<a href="https://colab.research.google.com/github/ciaoCiao100/ia_esame/blob/master/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -r requirements.txt

In [35]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sl
import seaborn as sns

from tensorflow import keras
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
import sklearn.metrics as metrics

# fa i modo che i numeri siano più leggibili.
np.set_printoptions(precision=3, suppress=True)

train_file_path =  "./train.csv"
test_file_path = "./test.csv"

train_df = pd.read_csv(train_file_path)
#train_df.dtypes
test_df = pd.read_csv(test_file_path)
#test_df.dtypes


eiminazione di valori Nan per il train

In [36]:
# Sostituisco ad ogni stringa vuota il valore 'NaN'
train_df.replace("", np.nan, inplace=True)
train_df.replace(" ", np.nan, inplace=True)
train_df.replace("?", np.nan, inplace=True)
train_df.head()

,No-show,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received
0,No,2.719167e+13,5730078,F,2016-05-24T08:07:44Z,2016-05-24T00:00:00Z,36,BELA VISTA,0,0,0,0,0,0
1,No,7.479664e+10,5685344,F,2016-05-11T10:11:49Z,2016-05-13T00:00:00Z,35,ITARARÉ,0,0,0,0,0,0
2,No,4.678419e+13,5738382,M,2016-05-25T10:51:57Z,2016-05-25T00:00:00Z,15,MARUÍPE,0,0,0,0,0,0
3,Yes,3.167173e+13,5667906,F,2016-05-06T08:15:22Z,2016-05-19T00:00:00Z,12,TABUAZEIRO,0,0,0,0,0,0
4,No,8.799566e+14,5751108,F,2016-05-31T09:41:47Z,2016-05-31T00:00:00Z,27,PRAIA DO SUÁ,0,0,0,0,0,0


controllo se ci sono effettivamente valori Nan nel dataset train 

In [37]:
train_df.isna().sum()


No-show           0
PatientId         0
AppointmentID     0
Gender            0
ScheduledDay      0
AppointmentDay    0
Age               0
Neighbourhood     0
Scholarship       0
Hipertension      0
Diabetes          0
Alcoholism        0
Handcap           0
SMS_received      0
dtype: int64

eliminazione di valori Nan per il test

In [38]:
# Sostituisco ad ogni stringa vuota il valore 'NaN'
test_df.replace("", np.nan, inplace=True)
test_df.replace(" ", np.nan, inplace=True)
test_df.replace("?", np.nan, inplace=True)
test_df.head()

,No-show,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received
0,No,4.759386e+12,5658195,M,2016-05-04T10:41:02Z,2016-05-04T00:00:00Z,9,TABUAZEIRO,0,0,0,0,0,0
1,Yes,7.287316e+13,5631991,F,2016-04-28T07:34:33Z,2016-05-02T00:00:00Z,49,ANDORINHAS,0,1,1,0,0,0
2,No,3.692358e+13,5672349,M,2016-05-09T07:56:53Z,2016-05-09T00:00:00Z,50,RESISTÊNCIA,0,0,0,0,0,0
3,No,5.363398e+14,5637581,M,2016-04-29T07:09:32Z,2016-05-03T00:00:00Z,8,SÃO PEDRO,0,0,0,0,0,1
4,Yes,6.673226e+11,5628498,F,2016-04-27T11:06:02Z,2016-05-11T00:00:00Z,14,CARATOÍRA,0,0,0,0,0,0


controllo se ci sono effettivamente valori Nan nel dataset test

In [39]:
test_df.isna().sum()

No-show           0
PatientId         0
AppointmentID     0
Gender            0
ScheduledDay      0
AppointmentDay    0
Age               0
Neighbourhood     0
Scholarship       0
Hipertension      0
Diabetes          0
Alcoholism        0
Handcap           0
SMS_received      0
dtype: int64

In [40]:
# Procediamo con l'eliminazione dei dati non validi (NaN)
test_df.dropna(inplace=True)
train_df.dropna(inplace=True)

conversione stringhe a float train.csv

In [41]:
#tutte le colonne che contengono stringhe devono essere trasformate in colonne che contengono interi 
for elemento in train_df:
  if elemento=="AppointmentID" or elemento=="Age" or elemento=="Scholarship" or elemento == "Hipertension	" or elemento == "Diabetes" or elemento == "Alcoholism" or elemento == "Handcap" or elemento == "SMS_received":
   train_df[elemento] = train_df[elemento].astype('float')
  else:
   train_df[elemento] = (pd.Categorical(train_df[elemento])).codes.astype('float')

# mostro il dataset dopo l'operazione
train_df.head()

,No-show,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received
0,0.0,22455.0,5730078.0,0.0,51818.0,17.0,36.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1639.0,5685344.0,0.0,35730.0,10.0,35.0,36.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,26073.0,5738382.0,1.0,54412.0,18.0,15.0,44.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,23157.0,5667906.0,0.0,28292.0,15.0,12.0,78.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,45044.0,5751108.0,0.0,58254.0,20.0,27.0,56.0,0.0,0.0,0.0,0.0,0.0,0.0


conversione da stringhe a float test.csv

In [42]:
#tutte le colonne che contengono stringhe devono essere trasformate in colonne che contengono interi 
for elemento in test_df:
   if elemento=="AppointmentID" or elemento=="Age" or elemento=="Scholarship" or elemento == "Hipertension	" or elemento == "Diabetes" or elemento == "Alcoholism" or elemento == "Handcap" or elemento == "SMS_received":
     test_df[elemento] = test_df[elemento].astype('float')
   else:
     test_df[elemento] = (pd.Categorical(test_df[elemento])).codes.astype('float')

# mostro il dataset dopo l'operazione
test_df.head()

,No-show,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received
0,0.0,4077.0,5658195.0,1.0,6311.0,3.0,9.0,76.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,10123.0,5631991.0,0.0,3739.0,1.0,49.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0
2,0.0,7994.0,5672349.0,1.0,7861.0,6.0,50.0,57.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,13498.0,5637581.0,1.0,4146.0,2.0,8.0,75.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1.0,1792.0,5628498.0,0.0,3510.0,8.0,14.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0


eliminazione colonne inutili train.csv

In [43]:
train_df = train_df.drop(['PatientId','ScheduledDay'], axis=1)
train_df.head()

,No-show,AppointmentID,Gender,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received
0,0.0,5730078.0,0.0,17.0,36.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,5685344.0,0.0,10.0,35.0,36.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,5738382.0,1.0,18.0,15.0,44.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,5667906.0,0.0,15.0,12.0,78.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,5751108.0,0.0,20.0,27.0,56.0,0.0,0.0,0.0,0.0,0.0,0.0


eliminazione colonne inutili test.csv

In [44]:
test_df = test_df.drop(['PatientId','ScheduledDay'], axis=1)
test_df.head()

,No-show,AppointmentID,Gender,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received
0,0.0,5658195.0,1.0,3.0,9.0,76.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,5631991.0,0.0,1.0,49.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0
2,0.0,5672349.0,1.0,6.0,50.0,57.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,5637581.0,1.0,2.0,8.0,75.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1.0,5628498.0,0.0,8.0,14.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0


preparazione modello 

In [45]:
features =['AppointmentID','Gender','AppointmentDay',	'Age','Neighbourhood','Scholarship','Hipertension','Diabetes','Alcoholism','Handcap','SMS_received']
X = train_df[features]
Y = train_df['No-show']
X_test = test_df[features]
Y_test =test_df['No-show']

# Specify Model : viene utilizzato RandomForestCLaasifier
#La foresta casuale, come suggerisce il nome, 
#è costituita da un gran numero di singoli alberi decisionali che operano come un insieme(differentemente per il DecisionTreeClassifier che opera su un albero di decisione).
#Ogni singolo albero nella foresta casuale genera una previsione di classe basandosi sui suoi attributi ed esprime un voto per quella classe
# e la classe con il maggior numero di voti diventa la previsione del nostro modello
#Mentre alcuni alberi potrebbero essere sbagliati, molti altri alberi avranno ragione, quindi come gruppo gli alberi sono in grado di muoversi nella direzione corretta. 
#Le previsioni (e quindi gli errori) fatte dai singoli alberi devono avere basse correlazioni tra loro.

model = RandomForestClassifier(random_state=1)
model.fit(X,Y)




RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)

addestramento

In [46]:
train_predictions = model.predict(X)
train_acc = metrics.accuracy_score(Y,train_predictions)
print('Accuracy train: %s'  % train_acc)

Accuracy train: 0.9999717258538792


test

In [47]:
test_predictions = model.predict(X_test)
test_acc=metrics.accuracy_score(Y_test,test_predictions)
print('Accuracy test: %s'  % test_acc)



Accuracy test: 0.7907831495617755


In [48]:
#print(classification_report(Y_test, test_predictions))